# Introdução ao **SCIKIT-LEARN**

O notebook treina um **classificador de Floresta Aleatória (Random Forest)** para prever se um paciente possui (ou não) doença cardíaca, usando um conjunto de dados tabular.

In [1]:
import pandas as pd

heart_disease_df = pd.read_csv("dataset/heart-disease.csv")

In [2]:
heart_disease_df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
# Remove a coluna target do DataFrame, o resultado só contém as features
x = heart_disease_df.drop("target", axis = 1)

# Seleciona apenas a coluna-alvo
y = heart_disease_df["target"]


No dialeto de ML, x é usualmente utilizado para o(s) feature(s) e y é convencionalmente usado como target.


In [4]:
# Importa o modelo Random Forest, que é um “conjunto” (ensemble) de árvores de decisão
from sklearn.ensemble import RandomForestClassifier

# Instância o classificador com hiper-parâmetros padrões (default)
clf = RandomForestClassifier()

# Imprime um dicionário com todos os hiper-parâmetros atuais - serve para ver ou registrar configurações
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## O que é uma árvore de decisão?

Imagine um fluxograma de perguntas sim/não que você percorre até chegar a uma conclusão. Cada pergunta usa uma característica (feature) dos dados; cada resposta leva a outro nó até que se alcance uma folha com o veredito.
Esse fluxograma é exatamente a representação de uma **árvore de decisão** em ML.

**Conceitos**: *Hiper-parâmetro* é aquilo definido antes do aprendizado. Já *Parâmetro* é o que o algoritmo aprende.

In [5]:
# Importa a função que embaralha e divide dados
from sklearn.model_selection import train_test_split

# train_test_split(...) quebra x e y em quatro partes (duas duplas: uma p/ treino e outra p/):
# 80% treino, 20% teste → test_size=0.2 == (0.2 * 100)%
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

## Por que dividir?

Para avaliar o modelo em dados nunca vistos e evitar “decorar” (overfitting).

In [6]:
clf.fit(x_train, y_train)

RandomForestClassifier()

O algoritmo vasculha os padrões em *x_train* e aprende como prever *y_train*.
Para floresta, isso significa construir várias árvores em subconjuntos de amostras e features.

In [7]:
y_preds = clf.predict(x_test) # predict gera rótulos 0/1 para cada exemplo do conjunto de teste

In [8]:
y_preds

array([0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1], dtype=int64)

## Por que gerar y_preds a partir do conjunto de teste e compará-lo com y_test?

### 1ª Etapa

y_preds = clf.predict(x_test) → O modelo, **sem ver as respostas corretas**, tenta adivinhar cada rótulo do conjunto de teste. → Isso simula o mundo real, onde o modelo recebe apenas features e precisa fazer uma previsão.

### 2ª Etapa

Comparar y_preds × y_test → Usamos métricas que colocam lado a lado "o que o modelo disse" (y_preds) e "o gabarito" (y_test). → Só assim sabemos quanto o modelo erra/acerta em dados "novos".

### 3ª Etapa

Guardar clf.predict(x_test) em variável (y_preds) → Armazenamos uma vez e reutilizamos para várias análises. → Evita recalcular previsões toda hora e facilita depuração.

In [9]:
clf.score(x_train, y_train)

1.0

Retorna acurácia sobre o próprio conjunto de treino. Valor perto de **1** pode indicar *overfitting* se o score de teste cair bastante.

## Regras de bolso para interpretar o gap treino vs. teste

| Gap treino – teste | Diagnóstico típico    | Ação sugerida                                                |
| ------------------ | --------------------- | ------------------------------------------------------------ |
| **≤ 5 p.p.**       | Modelo generaliza bem | OK                                                           |
| **5 – 10 p.p.**    | Leve overfitting      | Regularização leve, mais dados                               |
| **> 10 p.p.**      | Overfitting forte     | Poda/limitar profundidade, mais dados, features mais simples |

*p.p. = pontos percentuais (ex.: 0,95 → 0,85 = 10 p.p.).*

In [10]:
clf.score(x_test, y_test)

0.8852459016393442

Acurácia em dados "novos" (nosso termômetro de desempenho real).

O valor que sai de clf.score() ou accuracy_score() é um número objetivo (ex.: 0,75 = 75% de acerto). Mas decidir se 0,75 é *bom* ou *ruim* envolve fatores externos ao algoritmo.

In [11]:
# Importa três funções de avaliação / métricas
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# calcula precision, recall, f1-score e support para cada classe e imprime formatado
# aqui temos só duas classes, que no CSV eram 0 (sem doença) e 1 (com doença)
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86        24
           1       0.92      0.89      0.90        37

    accuracy                           0.89        61
   macro avg       0.88      0.88      0.88        61
weighted avg       0.89      0.89      0.89        61



## Como interpretar o output? 

### Linha da classe

| coluna        | “Tradução” em linguagem do dia-a-dia                                                                                    |
| ------------- | ----------------------------------------------------------------------------------------------------------------------- |
| **precision** | “De tudo que o modelo **acusou** como sendo desta classe, quantos estavam certos?”  →  **Qualidade** dos acertos.       |
| **recall**    | “Entre todos os exemplos que **realmente** são desta classe, quantos o modelo encontrou?” →  **Cobertura** dos acertos. |
| **f1-score**  | Média das duas métricas anteriores (um jeitinho de ter um único número).                                                         |
| **support**   | Quantas amostras dessa classe havia no teste → Serve só como referência.                                                |
### Linha Resumo 

| Linha                 | O que agrega                                        | Quando olhar                                                                               |
| --------------------- | --------------------------------------------------- | ------------------------------------------------------------------------------------------ |
| **accuracy**     | 80% das 61 previsões batem com o gabarito.         | Primeiro número global.                      |
| **macro avg**    | Média simples entre as classes (ignora “support”).  | Boa para saber se o modelo trata as classes por igual, mesmo se elas forem desbalanceadas. |
| **weighted avg** | Média que pesa cada classe pelo respectivo support. | Se as classes têm tamanhos bem diferentes, esse número é mais honesto que a macro.         |

### "Está Bom?"

- **Compare com o acaso**: se as classes fossem 50/50, um chute aleatório daria ~50% de accuracy. Seu 80% já é bem melhor.

- **Custo do erro**: se errar um doente (classe 1) for grave, concentre-se no recall da classe 1 (82 % ⇒ pegou 8 de cada 10 doentes).

- **Gap precision × recall**: na classe 0 eles estão próximos (84 × 79 %). Isso indica que o modelo não está pendendo muito para "chutar demais" ou "chutar de menos".

In [12]:
# Devolve matriz 2×2 com contagens: Verdadeiro Positivo, Falso Positivo, Verdadeiro Negativo, Falso Negativo.
confusion_matrix(y_test, y_preds)

array([[21,  3],
       [ 4, 33]], dtype=int64)

## Matriz de Confussão

|                           | **Realidade é POSITIVO**                                            | **Realidade é NEGATIVO**                                            |
| ------------------------- | ------------------------------------------------------------------- | ------------------------------------------------------------------- |
| **Modelo prevê POSITIVO** | **Verdadeiro Positivo (VP)**<br>- previu “positivo” e era mesmo.    | **Falso Positivo (FP)**<br>- previu “positivo”, mas era “negativo”. |
| **Modelo prevê NEGATIVO** | **Falso Negativo (FN)**<br>- previu “negativo”, mas era “positivo”. | **Verdadeiro Negativo (VN)**<br>- previu “negativo” e era mesmo.    |

## Fórmulas-chave

| Métrica                    | Fórmula                           |
| -------------------------- | --------------------------------- |
| **Accuracy (acurácia)**    | $(VP + VN) / (VP + FP + FN + VN)$ |
| **Precision (precisão)**   | $VP / (VP + FP)$                  |
| **Recall (sensibilidade)** | $VP / (VP + FN)$                  |

*Essas definições funcionam para qualquer problema binário.*

In [13]:
# Mesmo valor já visto em .score, mas via função externa (fora do scikit-lear)
accuracy_score(y_test, y_preds)

0.8852459016393442

In [14]:
import numpy as np # importa NumPy (essencial para listas numéricas)

np.random.seed(42) # toda vez que rodar o notebook, a sequência de "números aleatórios" será idêntica
# afeta o algorítmo em: train_test_split(...) → usa o gerador do NumPy para embaralhar os registros antes de cortas
#                       RandomForest → também depende deste gerador para sortear amostras e features para cada árvore

for i in range(10, 100, 10): # cria um laço de valores 
    print(f"Trying model eith {i} estimators...")
    model = RandomForestClassifier(n_estimators = i).fit(x_train, y_train) # a diferença do anterior era que antes, não
    # era passado nada em "n_estimators" → o scikit-learn usava o padrão de 100 árvores. Agora, cria-se uma floresta de 
    # decisão, só que desta vez é especificado manualmente o nº de árvores a cada laço (n_estimators = i).
    print(f"Model accuracy on test set: {model.score(x_test, y_test) * 100:.2f}%")
    print("")

Trying model eith 10 estimators...
Model accuracy on test set: 81.97%

Trying model eith 20 estimators...
Model accuracy on test set: 88.52%

Trying model eith 30 estimators...
Model accuracy on test set: 86.89%

Trying model eith 40 estimators...
Model accuracy on test set: 91.80%

Trying model eith 50 estimators...
Model accuracy on test set: 88.52%

Trying model eith 60 estimators...
Model accuracy on test set: 88.52%

Trying model eith 70 estimators...
Model accuracy on test set: 88.52%

Trying model eith 80 estimators...
Model accuracy on test set: 88.52%

Trying model eith 90 estimators...
Model accuracy on test set: 90.16%



# Conclusão

Com esse passo-a-passo, foi visto:

- Como os dados entram e saem no scikit-learn.

- Separação treino/teste e por que ela é crucial.

- Instanciação, ajuste manual de hiper-parâmetros, treino, predição e avaliação de um modelo supervisionado.

- Métricas básicas (acurácia, matriz de confusão, relatório de classificação).